In [1]:
import os

In [2]:
os.environ['HF_HOME'] = '/tmp/cache'

In [3]:
!pwd

/workspaces/LLMs--RAG/week2-opensourcellms


In [4]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          32G   12G   18G  40% /
tmpfs            64M     0   64M   0% /dev
shm              64M     0   64M   0% /dev/shm
/dev/root        29G   24G  5.7G  81% /vscode
/dev/sdb1        44G  100K   42G   1% /tmp
/dev/loop3       32G   12G   18G  40% /workspaces


In [7]:
!pwd

/home/jovyan/workspace/LLMs--RAG/week2-opensourcellms


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: unable to mmap 9449619912 bytes from file </tmp/cache/hub/models--google--flan-t5-xl/snapshots/7d6315df2c2fb742f0f5b556879d730926ca9001/model-00001-of-00002.safetensors>: Cannot allocate memory (12)

In [15]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

In [16]:
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie alt sind Sie?</s>
